In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
from wmb import cemba, mm10

from ALLCools.clustering import *
from ALLCools.mcds import MCDS
import pybedtools
from pyensembl import EnsemblRelease

In [2]:
# Parameters
chrom_to_remove = ["chrX", "chrY", "chrM", "chrL"]

## merge adata

In [67]:
import anndata as ad
big_adata = ad.concat([adata_1, adata_2],merge="same",keys=['Retina','Macula'])


## add annotation

## basic feature filtering

In [80]:
remove_chromosomes(new_big_adata, exclude_chromosomes=chrom_to_remove)

33062 regions remained.


In [ ]:
remove_black_list_region(new_big_adata, black_list_path='./hg38-blacklist.v2.bed')


In [85]:
# Convert the sparse matrix to a dense numpy array
dense_matrix = new_big_adata.X.toarray()
# Calculate standard deviation along the desired axis
pass_std_filter = np.std(dense_matrix, axis=0) > 0.05
# Subset the AnnData object
new_big_adata = new_big_adata[:, pass_std_filter]

In [88]:
n_cell = 5
if new_big_adata.n_obs > n_cell:
    filter_regions(new_big_adata, n_cell=n_cell, zscore_abs_cutoff=None)

18830 regions remained.


In [89]:
if new_big_adata.X.dtype != 'float32':
    new_big_adata.X = new_big_adata.X.astype('float32')

In [90]:
sc.pp.normalize_per_cell(new_big_adata)

In [91]:
sc.pp.log1p(new_big_adata)

In [92]:
std_cutoff= 0.05
new_big_adata.X = new_big_adata.X.toarray()
std_filter = new_big_adata.X.std(axis=0) > std_cutoff
new_big_adata._inplace_subset_var(std_filter)

In [94]:
# zero_center=False to prevent making dense matrix
sc.pp.scale(new_big_adata, zero_center=True)

In [95]:
new_big_adata.write_h5ad('big_rna_input.h5ad')

... storing 'annotation' as categorical
... storing 'chrom' as categorical
